In [1]:
from rag_module.rag import RAG
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\SjoerdO\AppData\Local\pypoetry\Cache\virtualenvs\rag--7y4UdRY-py3.13\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class explainaRAG(RAG):
    """
    This class extends the RAG class with explanation methods.
    """

    def __init__(self, *args, **kwargs):
        """
        Initializes the explainaRAG instance with the provided arguments.
        """
        super().__init__(*args, **kwargs)

    def is_in_scope(self, query, threshold=0.5):
        n_chunks = self.embedder.index.ntotal
        n_dims = self.embedder.dim
        
        document_vectors = np.zeros((n_chunks, n_dims), dtype=np.float32)
        for i in range(n_chunks):
            document_vectors[i] = self.embedder.index.reconstruct(i)
        query_vector = self.embedder.encode(query).reshape(1, -1)
        similarities = cosine_similarity(query_vector, document_vectors)
        
        max_sim = np.max(similarities)  # Highest similarity score
        if max_sim > threshold:
            return True, max_sim
        else:
            return False, max_sim
        

In [18]:
rag = explainaRAG()

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Loading default flashrank model for language en
Default Model: ms-marco-MiniLM-L-12-v2
Loading FlashRankRanker model ms-marco-MiniLM-L-12-v2 (this message can be suppressed by setting verbose=0)
Loading model FlashRank model ms-marco-MiniLM-L-12-v2...


In [19]:
query = "What is the capital of France?"
rag.is_in_scope(query)

Batches: 100%|██████████| 1/1 [00:00<00:00, 42.79it/s]


(False, np.float32(0.17998654))

# In/Out scope

In [3]:
import faiss
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Load FAISS index
faiss_index_path = "./embeddings/guidance_framework_2/chunk_vectors.faiss"
index = faiss.read_index(faiss_index_path)

# Get stored document vectors
num_docs = index.ntotal
d = index.d  # Vector dimension
document_vectors = np.zeros((num_docs, d), dtype=np.float32)
for i in range(num_docs):
    document_vectors[i] = index.reconstruct(i)

# Load embedding model (use the same one used to create FAISS index)
model = SentenceTransformer("all-MiniLM-L6-v2")

# Embed user query
query_text = "explainability in RAG systems."
query_vector = model.encode(query_text).astype(np.float32).reshape(1, -1)

# Search in FAISS (find nearest neighbors)
k = 5  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_vector, k)

# Compute cosine similarity
similarities = cosine_similarity(query_vector, document_vectors)
max_sim = np.max(similarities)  # Highest similarity score
threshold = 0.5  # Define a threshold for "in scope"

# Reduce dimensions for visualization using PCA
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(np.vstack([document_vectors, query_vector]))

# Split transformed vectors
docs_2d = reduced_vectors[:-1]  # Document embeddings
query_2d = reduced_vectors[-1]   # Query embedding

# Plot document space
#plt.scatter(docs_2d[:, 0], docs_2d[:, 1], label="Documents", alpha=0.5, s=5)
#plt.scatter(query_2d[0], query_2d[1], color='orange', label="User Query", s=20)

# # Mark if query is out-of-scope
# color = "green" if max_sim > threshold else "red"
# plt.text(query_2d[0], query_2d[1], "OUT-OF-SCOPE" if max_sim < threshold else "IN-SCOPE", 
#          color=color, fontsize=12, ha='right')

#plt.xlabel("Component 1")
#plt.ylabel("Component 2")
#plt.title("RAG Feature Space - Query vs Database")
#plt.legend()
#plt.show()

# Print results
if max_sim > threshold:
    print(f"✅ Query: {query_text}\n is IN-SCOPE (Max similarity: {max_sim:.2f})")
else:
    print(f"❌ Query: {query_text}\n is OUT-OF-SCOPE (Max similarity: {max_sim:.2f})")


❌ Query: explainability in RAG systems.
 is OUT-OF-SCOPE (Max similarity: 0.33)
